## Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

In [127]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

stocks = pd.read_csv('data/sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW', 'CTL', "ETFC", "MYL", "NBL"])]
from secrets import IEX_CLOUD_API_TOKEN

In [128]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2574370895316,
 'week52high': 182.94,
 'week52low': 129.65,
 'week52highSplitAdjustOnly': 191.72,
 'week52lowSplitAdjustOnly': 129.86,
 'week52change': 0.0461037216351294,
 'sharesOutstanding': 16146000643,
 'float': 0,
 'avg10Volume': 102452097,
 'avg30Volume': 89109609,
 'day200MovingAvg': 152.53,
 'day50MovingAvg': 152.29,
 'employees': 149694,
 'ttmEPS': 6.14,
 'ttmDividendRate': 0.9273229097078595,
 'dividendYield': 0.005827547676400629,
 'nextDividendDate': '2022-10-21',
 'exDividendDate': '2022-08-02',
 'nextEarningsDate': '2023-01-17',
 'peRatio': 25.756372300582836,
 'beta': 1.2563547227710818,
 'maxChangePercent': 61.88031311111723,
 'year5ChangePercent': 3.1619066501359,
 'year2ChangePercent': 0.3679627747368057,
 'year1ChangePercent': 0.0470970487214504,
 'ytdChangePercent': -0.12318265591889797,
 'month6ChangePercent': -0.00962450766082632,
 'month3ChangePercent': -0.042279046603103526,
 'month1ChangePercent': 0.09401962945310345,

## Parsing our first API call 

In [129]:
data['year1ChangePercent']

0.0470970487214504

In [130]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [131]:
final_dataframe = pd.DataFrame(columns = my_columns)


for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,144.58,-0.115252,N/A
1,AAL,14.00,-0.273492,N/A
2,AAP,194.49,-0.134479,N/A
3,AAPL,157.14,0.046164,N/A
4,ABBV,148.81,0.348549,N/A
...,...,...,...,...
492,YUM,120.50,-0.040349,N/A
493,ZBH,115.63,-0.209469,N/A
494,ZBRA,293.00,-0.481670,N/A
495,ZION,52.65,-0.167365,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [132]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,81.13,2.314725,N/A
1,OXY,72.11,1.170690,N/A
2,DVN,79.05,1.017672,N/A
3,CF,109.33,0.935497,N/A
4,MCK,398.13,0.930903,N/A
5,HRB,41.66,0.909296,N/A
6,UNM,46.12,0.896462,N/A
7,MRO,31.42,0.885065,N/A
8,XOM,111.80,0.814142,N/A
9,COP,128.82,0.811738,N/A


In [133]:
# calculating the number of shares to buy
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000
1000000


In [134]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

C:\Users\octav\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,81.13,2.314725,241
1,OXY,72.11,1.170690,271
2,DVN,79.05,1.017672,248
3,CF,109.33,0.935497,179
4,MCK,398.13,0.930903,49
5,HRB,41.66,0.909296,470
6,UNM,46.12,0.896462,425
7,MRO,31.42,0.885065,624
8,XOM,111.80,0.814142,175
9,COP,128.82,0.811738,152


## Improving the momentum strategy 
Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

    High-quality momentum stocks show "slow and steady" outperformance over long periods of time
    Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of:

    1-month price returns
    3-month price returns
    6-month price returns
    1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation hqm often. It stands for high-quality momentum.

In [135]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], index = hqm_columns), ignore_index = True)
        
hqm_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy,HQM Score,One-Month Price Return,One-Month Return Percentile,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile
0,A,138.82,-0.116495,N/A,N/A,0.136016,N/A,N/A,0.168514,N/A,0.038164,N/A
1,AAL,14.00,-0.271558,N/A,N/A,0.147580,N/A,N/A,-0.255322,N/A,0.021997,N/A
2,AAP,198.78,-0.132502,N/A,N/A,0.201342,N/A,N/A,-0.028003,N/A,-0.006705,N/A
3,AAPL,160.57,0.045609,N/A,N/A,0.096432,N/A,N/A,-0.009574,N/A,-0.040422,N/A
4,ABBV,149.65,0.351643,N/A,N/A,0.046416,N/A,N/A,0.025677,N/A,0.039997,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
492,YUM,123.16,-0.039901,N/A,N/A,0.102999,N/A,N/A,0.017988,N/A,-0.034555,N/A
493,ZBH,117.64,-0.211717,N/A,N/A,0.061939,N/A,N/A,-0.057083,N/A,0.030414,N/A
494,ZBRA,291.00,-0.479630,N/A,N/A,0.085238,N/A,N/A,-0.229226,N/A,-0.197246,N/A
495,ZION,53.99,-0.161838,N/A,N/A,-0.004850,N/A,N/A,-0.075386,N/A,-0.050478,N/A


## Calculating momentum percetiles
We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

    One-Year Price Return
    Six-Month Price Return
    Three-Month Price Return
    One-Month Price Return


In [136]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

# filter out any of the null values in Price Return
hqm_dataframe[~hqm_dataframe['One-Year Price Return'].isnull()]
hqm_dataframe[~hqm_dataframe['Six-Month Price Return'].isnull()]
hqm_dataframe[~hqm_dataframe['Three-Month Price Return'].isnull()]
hqm_dataframe[~hqm_dataframe['One-Month Price Return'].isnull()]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])

        
# # Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

0      49.295775
1      22.535211
2      46.076459
3      72.837022
4      90.342052
         ...    
492    59.557344
493    32.595573
494     4.627767
495    39.839034
496    19.315895
Name: One-Year Return Percentile, Length: 497, dtype: object
0      87.122736
1        9.45674
2       53.31992
3      58.350101
4      67.404427
         ...    
492    65.995976
493    44.466801
494    12.072435
495    41.046278
496    29.376258
Name: Six-Month Return Percentile, Length: 497, dtype: object
0      69.617706
1      67.203219
2      56.539235
3      48.088531
4      70.221328
         ...    
492    49.899396
493    68.209256
494      8.04829
495    44.064386
496      13.8833
Name: Three-Month Return Percentile, Length: 497, dtype: object
0      69.215292
1      73.641851
2      89.738431
3      53.722334
4      29.577465
         ...    
492    56.338028
493    37.424547
494    47.283702
495    14.285714
496    21.529175
Name: One-Month Return Percentile, Length: 497, dtype: object


In [137]:
hqm_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy,HQM Score,One-Month Price Return,One-Month Return Percentile,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile
0,A,138.82,-0.116495,N/A,N/A,0.136016,69.215292,49.295775,0.168514,87.122736,0.038164,69.617706
1,AAL,14.00,-0.271558,N/A,N/A,0.147580,73.641851,22.535211,-0.255322,9.45674,0.021997,67.203219
2,AAP,198.78,-0.132502,N/A,N/A,0.201342,89.738431,46.076459,-0.028003,53.31992,-0.006705,56.539235
3,AAPL,160.57,0.045609,N/A,N/A,0.096432,53.722334,72.837022,-0.009574,58.350101,-0.040422,48.088531
4,ABBV,149.65,0.351643,N/A,N/A,0.046416,29.577465,90.342052,0.025677,67.404427,0.039997,70.221328
...,...,...,...,...,...,...,...,...,...,...,...,...
492,YUM,123.16,-0.039901,N/A,N/A,0.102999,56.338028,59.557344,0.017988,65.995976,-0.034555,49.899396
493,ZBH,117.64,-0.211717,N/A,N/A,0.061939,37.424547,32.595573,-0.057083,44.466801,0.030414,68.209256
494,ZBRA,291.00,-0.479630,N/A,N/A,0.085238,47.283702,4.627767,-0.229226,12.072435,-0.197246,8.04829
495,ZION,53.99,-0.161838,N/A,N/A,-0.004850,14.285714,39.839034,-0.075386,41.046278,-0.050478,44.064386


## Calculating the HQM Score

We'll now calculate our HQM Score, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The HQM Score will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [138]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [140]:
hqm_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy,HQM Score,One-Month Price Return,One-Month Return Percentile,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile
0,A,138.82,-0.116495,N/A,68.812877,0.136016,69.215292,49.295775,0.168514,87.122736,0.038164,69.617706
1,AAL,14.00,-0.271558,N/A,43.209256,0.147580,73.641851,22.535211,-0.255322,9.45674,0.021997,67.203219
2,AAP,198.78,-0.132502,N/A,61.418511,0.201342,89.738431,46.076459,-0.028003,53.31992,-0.006705,56.539235
3,AAPL,160.57,0.045609,N/A,58.249497,0.096432,53.722334,72.837022,-0.009574,58.350101,-0.040422,48.088531
4,ABBV,149.65,0.351643,N/A,64.386318,0.046416,29.577465,90.342052,0.025677,67.404427,0.039997,70.221328
...,...,...,...,...,...,...,...,...,...,...,...,...
492,YUM,123.16,-0.039901,N/A,57.947686,0.102999,56.338028,59.557344,0.017988,65.995976,-0.034555,49.899396
493,ZBH,117.64,-0.211717,N/A,45.674044,0.061939,37.424547,32.595573,-0.057083,44.466801,0.030414,68.209256
494,ZBRA,291.00,-0.479630,N/A,18.008048,0.085238,47.283702,4.627767,-0.229226,12.072435,-0.197246,8.04829
495,ZION,53.99,-0.161838,N/A,34.808853,-0.004850,14.285714,39.839034,-0.075386,41.046278,-0.050478,44.064386


In [155]:
# Selecting the 50 best momentum stocks
hqm_dataframe=hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe=hqm_dataframe.reset_index()
hqm_dataframe = hqm_dataframe[:51]
hqm_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy,HQM Score,One-Month Price Return,One-Month Return Percentile,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile
0,42,APA,47.10,0.776573,416,93.61167,0.318725,98.390342,97.78672,0.121788,82.49497,0.232699,95.774648
1,33,AMGN,274.11,0.380059,71,89.486922,0.203605,90.342052,91.750503,0.196997,90.140845,0.119407,85.714286
2,34,AMP,327.56,0.052886,59,87.323944,0.233274,94.366197,74.044266,0.187771,89.738431,0.169313,91.146881
3,11,ADM,99.34,0.525399,197,86.368209,0.185811,85.714286,94.768612,0.070051,75.653924,0.155919,89.336016
4,16,AES,27.49,0.083410,713,86.217304,0.169238,80.885312,75.050302,0.306036,95.573441,0.190846,93.360161
5,17,AFL,65.84,0.240850,297,83.853119,0.151002,74.849095,87.525151,0.147854,85.714286,0.141089,87.323944
6,5,ABC,160.56,0.334896,122,82.344064,0.172055,82.293763,90.140845,0.060590,74.245473,0.096373,82.696177
7,23,ALB,281.89,0.127751,69,75.503018,0.056558,34.607646,79.476861,0.478396,98.993964,0.151493,88.933602
8,21,AJG,197.44,0.140754,99,74.346076,0.105697,57.746479,80.885312,0.127238,83.098592,0.058668,75.653924
9,18,AIG,59.20,-0.016024,331,72.334004,0.197095,87.927565,61.368209,-0.017635,56.136821,0.103679,83.903421


In [146]:
# calculating the number of shares to buy
portfolio_input()

Enter the value of your portfolio:1000000


In [147]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker']) - 1):
    hqm_dataframe.loc[i, "Number of Shares to Buy"] = math.floor(position_size / hqm_dataframe['Price'][i])

C:\Users\octav\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [148]:
hqm_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy,HQM Score,One-Month Price Return,One-Month Return Percentile,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile
0,A,138.82,-0.116495,141,68.812877,0.136016,69.215292,49.295775,0.168514,87.122736,0.038164,69.617706
1,AAL,14.00,-0.271558,1400,43.209256,0.147580,73.641851,22.535211,-0.255322,9.45674,0.021997,67.203219
2,AAP,198.78,-0.132502,98,61.418511,0.201342,89.738431,46.076459,-0.028003,53.31992,-0.006705,56.539235
3,AAPL,160.57,0.045609,122,58.249497,0.096432,53.722334,72.837022,-0.009574,58.350101,-0.040422,48.088531
4,ABBV,149.65,0.351643,131,64.386318,0.046416,29.577465,90.342052,0.025677,67.404427,0.039997,70.221328
5,ABC,160.56,0.334896,122,82.344064,0.172055,82.293763,90.140845,0.060590,74.245473,0.096373,82.696177
6,ABMD,265.15,-0.228424,73,29.275654,0.037288,27.7666,29.577465,-0.103260,35.01006,-0.123941,24.748491
7,ABT,100.75,-0.215258,194,30.331992,0.017698,22.535211,31.38833,-0.116548,32.796781,-0.085377,34.607646
8,ACN,301.74,-0.187475,64,48.289738,0.120961,63.179074,36.418511,-0.034905,51.10664,-0.057784,42.454728
9,ADBE,340.60,-0.502777,57,28.169014,0.175702,83.098592,3.018109,-0.180402,19.71831,-0.207833,6.841046


In [156]:
# Formatting Our Excel Output
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [157]:
# Creating the formats we'll need for our excel file
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [159]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)



In [160]:
# Save the Excel
writer.save()